In [ ]:
print("hola")

In [ ]:
import tensorflow as tf
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
print(tf.__version__)

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2
# %matplotlib notebook

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
batch1 = unpickle('/home/computadora1/Escritorio/VAE/cifar-10-batches-py/data_batch_1')
batch2 = unpickle('/home/computadora1/Escritorio/VAE/cifar-10-batches-py/data_batch_2')
batch3 = unpickle('/home/computadora1/Escritorio/VAE/cifar-10-batches-py/data_batch_3')
batch4 = unpickle('/home/computadora1/Escritorio/VAE/cifar-10-batches-py/data_batch_4')
batch5 = unpickle('/home/computadora1/Escritorio/VAE/cifar-10-batches-py/data_batch_5')

for a in batch1:
    print(a)

In [ ]:
images = []

In [ ]:
dataset = batch5[b'data']
for arr in dataset:
#     img_r = arr[:1024].reshape((32,32))
#     img_g = arr[1024:-1024].reshape((32,32))
#     img_b = arr[-1024:].reshape((32,32))
#     img_rgb = cv2.merge([img_r, img_g, img_b])
    images.append(arr)

In [ ]:
Dataset = np.ndarray(shape=(50000,3072), dtype=np.uint8)
Dataset[:] = images[:]
#Dataset = np.ndarray((images))
#print(Dataset.shape)
Dataset.shape

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis

def mnist_grid(X):
    X = X.reshape([-1,3072])
    num_images = X.shape[0]
    fig = figure()
    fig.set_size_inches(10,10)
    
    for i in range(num_images):
        a = fig.add_subplot(1, num_images, i+1)
        img_r = X[i,:1024].reshape((32,32))
        img_g = X[i,1024:-1024].reshape((32,32))
        img_b = X[i,-1024:].reshape((32,32))
        img_rgb = cv2.merge([img_r, img_g, img_b])
        imshow(img_rgb)
        axis('off')
    
    plt.show()

In [ ]:
X_train = Dataset[:40000]
X_valid = Dataset[-10000:]
mnist_grid(X_train[:10])
print(X_train.shape)
print(X_valid.shape)

In [ ]:
tf.reset_default_graph()

n_inputs = X_train.shape[1] # 3072
n_hidden1 = 512
n_hidden2 = 256
n_hidden3 = 128
n_hidden4 = 32
n_hidden5 = n_hidden3
n_hidden6 = n_hidden2
n_hidden7 = n_hidden1
n_outputs = n_inputs

x = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.elu)
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu)
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.elu)
hidden4_mean = tf.layers.dense(hidden3, n_hidden4, activation=None)
hidden4_sigma = tf.layers.dense(hidden3, n_hidden4, activation=None)
noise = tf.random_normal(tf.shape(hidden4_sigma), dtype=tf.float32)
hidden4 = hidden4_mean + hidden4_sigma * noise
hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.elu)
hidden6 = tf.layers.dense(hidden5, n_hidden6, activation=tf.nn.elu)
hidden7 = tf.layers.dense(hidden6, n_hidden7, activation=tf.nn.elu)
logits = tf.layers.dense(hidden7, n_outputs, activation=None)
outputs = tf.sigmoid(logits)

# MSE loss
def lossFunction(X):
    mse_loss = tf.reduce_mean(tf.reduce_sum(tf.square(X - outputs), axis=1))
    return mse_loss

#
# Funcion de perdida (MSE) y entrenamiento
#
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=logits)
xentropy_loss = tf.reduce_sum(xentropy)
eps = 1e-10
latent_loss = 0.5 * tf.reduce_sum(
    # centre en 0 y los demas los centre a 1
    tf.square(hidden4_sigma) + tf.square(hidden4_mean) - 1 - tf.log(eps + tf.square(hidden4_sigma))
) 

loss = xentropy_loss + latent_loss

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

loss_training = lossFunction(X_train) 
loss_validation = lossFunction(X_valid)

saver = tf.train.Saver()

In [ ]:
n_iterations = 101

# points = []
# points_train = []
# points_valid = []

# fig = plt.figure()
# ax = fig.add_subplot(111)

# plt.ion()

# fig.show()
# fig.canvas.draw()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
        
        _, mse_train = sess.run([training_op, loss_training], feed_dict={x: X_train})
#        mse_valid = sess.run([loss_validation],feed_dict={x:X_valid})
                
#         points_train.append(mse_train)
#         points_valid.append(mse_valid)
        
#         points = np.arange(iteration + 1)
        
#         ax.clear()
        
#         ax.plot(points, points_train, color='blue')
#         ax.plot(points, points_valid, color='red')
#         fig.canvas.draw()
        #time.sleep(1.0)
        #plt.show()
        
        if iteration % 50 == 0:
            #mse_valid = sess.run([loss_validation],feed_dict={x:X_valid})

            print('Iteracion: {:04d}'.format(iteration))
            print('MSE_train: {:.1f}'.format(mse_train))
            #print('MSE_valid: {:.9f}'.format(mse_valid[0]))
            ##print(mse_valid)
            ##print('Iteración: {:04d}, MSE: {:.9f}'.format(iteration, mse))
            R = sess.run(outputs, feed_dict={x:X_train[:10]})
            mnist_grid(R)
            
    saver.save(sess, "./VAE_2.ckpt")
    
    #R = sess.run(outputs, feed_dict={x:X_valid[:10]})
    #mnist_grid(R)

In [ ]:
n_digits = 10
random_codings = np.random.normal(size=[n_digits, n_hidden4])

with tf.Session() as sess:
  saver.restore(sess, "./VAE_2.ckpt")
  generated_images = sess.run(outputs, feed_dict={hidden4: random_codings})

mnist_grid(generated_images)

In [ ]:
points_valid[-10:]

points = np.arange(len(points_train))
plt.title('Reconstruction MNIST images 2D - VAE Dense Layers')
plt.ylabel('MSE - Loss')
plt.xlabel('Iterations')
plt.plot(points, points_train, label='train')
plt.plot(points, points_valid, label='test')
plt.legend()

plt.show()